# Práctica 0: Implementación de Perceptrón Multicapa desde Cero

Implementación de un perceptrón multicapa (MLP) sin usar frameworks de deep learning, solo NumPy.

**Objetivos:**
- Implementar forward pass y backpropagation manualmente
- Probar con XOR (problema no linealmente separable)
- Aplicar a problema de regresión

In [ ]:
# Configuración inicial
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import time
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

np.random.seed(42)
plt.style.use('seaborn-v0_8')

print("Configuración inicial completada")

In [ ]:
# Implementación del Perceptrón Multicapa
class MultiLayerPerceptron:
    def __init__(self, input_size, hidden_size, output_size, 
                 learning_rate=0.01, epochs=5000, activation='relu', task='classification'):
        # Inicialización Xavier
        xavier_std = np.sqrt(2.0 / (input_size + hidden_size))
        self.W1 = np.random.normal(0, xavier_std, (input_size, hidden_size))
        self.b1 = np.zeros((1, hidden_size))
        
        xavier_std2 = np.sqrt(2.0 / (hidden_size + output_size))
        self.W2 = np.random.normal(0, xavier_std2, (hidden_size, output_size))
        self.b2 = np.zeros((1, output_size))
        
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.activation = activation
        self.task = task
        
        self.train_errors = []
        self.validation_errors = []
    
    def _activation_function(self, x):
        if self.activation == 'relu':
            return np.maximum(0, x)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-np.clip(x, -250, 250)))
        elif self.activation == 'tanh':
            return np.tanh(x)
    
    def _activation_derivative(self, x):
        if self.activation == 'relu':
            return (x > 0).astype(float)
        elif self.activation == 'sigmoid':
            return x * (1 - x)
        elif self.activation == 'tanh':
            return 1 - x ** 2
    
    def _output_function(self, x):
        if self.task == 'classification':
            return 1 / (1 + np.exp(-np.clip(x, -250, 250)))
        else:
            return x
    
    def _output_derivative(self, x):
        if self.task == 'classification':
            return x * (1 - x)
        else:
            return np.ones_like(x)
    
    def forward(self, X):
        # Capa oculta
        self.z1 = np.dot(X, self.W1) + self.b1
        self.a1 = self._activation_function(self.z1)
        
        # Capa de salida
        self.z2 = np.dot(self.a1, self.W2) + self.b2
        self.output = self._output_function(self.z2)
        
        return self.output
    
    def backward(self, X, y):
        m = X.shape[0]
        
        # Error capa de salida
        error = y - self.output
        d_output = error * self._output_derivative(self.output)
        
        # Error capa oculta
        error_hidden = d_output.dot(self.W2.T)
        d_hidden = error_hidden * self._activation_derivative(self.a1)
        
        # Actualizar pesos
        self.W2 += (self.a1.T.dot(d_output) / m) * self.learning_rate
        self.b2 += np.mean(d_output, axis=0, keepdims=True) * self.learning_rate
        self.W1 += (X.T.dot(d_hidden) / m) * self.learning_rate
        self.b1 += np.mean(d_hidden, axis=0, keepdims=True) * self.learning_rate
    
    def fit(self, X, y, X_val=None, y_val=None, verbose=True):
        start_time = time.time()
        
        for epoch in range(self.epochs):
            self.forward(X)
            self.backward(X, y)
            
            # Registrar errores
            mse_train = np.mean((y - self.output) ** 2)
            self.train_errors.append(mse_train)
            
            if X_val is not None and y_val is not None):
                val_pred = self.forward(X_val)
                mse_val = np.mean((y_val - val_pred) ** 2)
                self.validation_errors.append(mse_val)
            
            # Mostrar progreso
            if verbose and (epoch % (self.epochs // 10) == 0 or epoch == self.epochs - 1):
                if X_val is not None:
                    print(f"Época {epoch:4d}: Train MSE = {mse_train:.6f}, Val MSE = {mse_val:.6f}")
                else:
                    print(f"Época {epoch:4d}: MSE = {mse_train:.6f}")
        
        return time.time() - start_time
    
    def predict(self, X):
        output = self.forward(X)
        if self.task == 'classification':
            return (output > 0.5).astype(int)
        else:
            return output

print("Clase MultiLayerPerceptron definida")